In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/tpch.csv', sep=',')

In [4]:
df.shape

(6001215, 14)

In [7]:
df.head()

,id,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,sum_qty,count_order,p_size,ps_min_supplycost,revenue,o_totalprice,o_shippriority
0,1,97616494.46,9.281309e+07,9.655614e+07,25.235616,38206.064368,0.049123,64477,2555,31,54.93,33956.0043,157152.37,0
1,2,48267290.15,4.581517e+07,4.759172e+07,25.853344,38893.867969,0.050274,32084,1241,48,154.33,5938.0480,37722.25,0
2,3,95371945.77,9.062359e+07,9.420106e+07,25.687650,38240.555642,0.050449,64065,2494,15,311.91,5711.1021,93733.64,0
3,4,92278812.00,8.764915e+07,9.120596e+07,25.437733,38179.069921,0.050017,61483,2417,23,196.92,22395.7578,144832.34,0
4,5,96048961.61,9.129832e+07,9.496757e+07,25.715611,38745.043005,0.049734,63749,2479,34,14.81,13359.2515,123773.56,0


In [12]:
df1 = df.head(1000000)

In [17]:
numRows, numCols = df1.shape

In [14]:
df1.head()

,id,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,sum_qty,count_order,p_size,ps_min_supplycost,revenue,o_totalprice,o_shippriority
0,1,97616494.46,9.281309e+07,9.655614e+07,25.235616,38206.064368,0.049123,64477,2555,31,54.93,33956.0043,157152.37,0
1,2,48267290.15,4.581517e+07,4.759172e+07,25.853344,38893.867969,0.050274,32084,1241,48,154.33,5938.0480,37722.25,0
2,3,95371945.77,9.062359e+07,9.420106e+07,25.687650,38240.555642,0.050449,64065,2494,15,311.91,5711.1021,93733.64,0
3,4,92278812.00,8.764915e+07,9.120596e+07,25.437733,38179.069921,0.050017,61483,2417,23,196.92,22395.7578,144832.34,0
4,5,96048961.61,9.129832e+07,9.496757e+07,25.715611,38745.043005,0.049734,63749,2479,34,14.81,13359.2515,123773.56,0


In [19]:
from pulp import *

### First Query ####

In [20]:
lp_variables = []
for i in range(numRows):
    lp_variables.append(LpVariable("t_{}".format(i+1), 0, 1))

In [26]:
prob = LpProblem("firstQuery", LpMaximize)

In [27]:
prob += lpDot(lp_variables, df1['count_order'])

In [28]:
constraint1 = (lpDot(lp_variables, df1['sum_base_price']) <= 15469853.7043)

In [29]:
constraint2 = (lpDot(lp_variables, df1['sum_disc_price']) <= 45279795.0584)

In [30]:
constraint3 = (lpDot(lp_variables, df1['sum_charge']) <= 95250227.7918)

In [31]:
constraint4 = (lpDot(lp_variables, df1['avg_qty']) <= 50.353948653)

In [32]:
constraint5 = (lpDot(lp_variables, df1['avg_price']) <= 68677.5852459)

In [33]:
constraint6 = (lpDot(lp_variables, df1['avg_disc']) <= 0.110243522496)

In [34]:
constraint7 = (lpDot(lp_variables, df1['sum_qty']) <= 77782.028739)

In [35]:
constraint8 = (lpSum(lp_variables) >= 1)

In [36]:
prob += constraint1
prob += constraint2
prob += constraint3
prob += constraint4
prob += constraint5
prob += constraint6
prob += constraint7
prob += constraint8

In [37]:
status = prob.solve()

In [39]:
LpStatus[status]

'Optimal'

In [40]:
ans = [value(v) for v in lp_variables]

In [46]:
for i, val in enumerate(ans):
    if val > 0:
        print(i, val)

100972 0.6035174
139394 1.0
818569 0.52891753


### Second Query ###

In [48]:
prob2 = LpProblem("secondQuery", LpMinimize)
lp_variables = []
for i in range(numRows):
    lp_variables.append(LpVariable("t_{}".format(i+1), 0, 1))

objective_prob = lpDot(lp_variables, df1['ps_min_supplycost'])

constraint1 = (lpDot(lp_variables, df1['p_size']) <= 8)
constraint2 = (lpSum(lp_variables) >= 1)
prob2 += objective_prob
prob2 += constraint1
prob2 += constraint2
status1 = prob2.solve()
print(LpStatus[status1])

Optimal


In [49]:
ans = [value(v) for v in lp_variables]
for i, val in enumerate(ans):
    if val > 0:
        print(i, val)

112677 1.0


#### Third Query ####

In [50]:
prob3 = LpProblem("thirdQuery", LpMinimize)
lp_variables = []
for i in range(numRows):
    lp_variables.append(LpVariable("t_{}".format(i+1), 0, 1))

objective_prob = lpSum(lp_variables)

constraint1 = (lpDot(lp_variables, df1['revenue']) >= 413930.849506)
constraint2 = (lpSum(lp_variables) >= 1)
prob3 += objective_prob
prob3 += constraint1
prob3 += constraint2
status3 = prob3.solve()
print(LpStatus[status3])
ans = [value(v) for v in lp_variables]
for i, val in enumerate(ans):
    if val > 0:
        print(i, val)

Optimal
9670 1.0
572518 0.95042532


### Forth Query ####

In [51]:
prob4 = LpProblem("forthQuery", LpMinimize)
lp_variables = []
for i in range(numRows):
    lp_variables.append(LpVariable("t_{}".format(i+1), 0, 1))

objective_prob = lpSum(lp_variables)

constraint1 = (lpDot(lp_variables, df1['o_totalprice']) <= 453998.242103)
constraint2 = (lpDot(lp_variables, df1['o_shippriority']) >= 3)
constraint3 = (lpSum(lp_variables) >= 1)
prob4 += objective_prob
prob4 += constraint1
prob4 += constraint2
prob4 += constraint3
status4 = prob4.solve()
print(LpStatus[status4])
ans = [value(v) for v in lp_variables]
for i, val in enumerate(ans):
    if val > 0:
        print(i, val)

Optimal
399797 1.0
630712 0.5
